# SQL 작성

In [1]:
import sys
import os
import pandas as pd
import cv2
import pickle
import json
sys.path.append('../../src')
import DISData as DD

## SQL 연결

In [2]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
# query 실행
query1 = (f'''CREATE TABLE IF NOT EXISTS RIT_Eyes_s_general_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        camera_3d_center VARCHAR(200) NOT NULL,
        camera_distance VARCHAR(200) NOT NULL,
        iris_loc VARCHAR(200) NOT NULL,
        eye_loc VARCHAR(200) NOT NULL,
        gaze_angle_az VARCHAR(200) NOT NULL,
        gaze_angle_el VARCHAR(200) NOT NULL,
        ortho_scale VARCHAR(200) NOT NULL,
        cam_az VARCHAR(200) NOT NULL,
        cam_el VARCHAR(200) NOT NULL,
        glasses VARCHAR(200) NOT NULL,
        pupil VARCHAR(200) NOT NULL,
        eye_lid VARCHAR(200) NOT NULL,
        iris_rot VARCHAR(200) NOT NULL,
        left_corner VARCHAR(200) NOT NULL,
        right_corner VARCHAR(200) NOT NULL,
        sclera VARCHAR(200) NOT NULL)''')

In [4]:
query2 = (f'''INSERT INTO RIT_Eyes_s_general_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, camera_3d_center, camera_distance,
          iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
          glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

## query 실행

In [5]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [6]:
label_path = []
label_name= []

# label 경로
label_path = 'Z:\\RIT_Eyes_s_general\\data\\labeldata'
image_path = 'Z:\\RIT_Eyes_s_general\\data\\rawdata'

value_list1 = []
error_list = []
count = 0

data_names = os.listdir(image_path)
for data_name in data_names:
    image_path1 = os.path.join(image_path,data_name) 
    for accesory in os.listdir(image_path1):
        image_path2 = os.path.join(image_path1,accesory)
        for image_name in os.listdir(image_path2):
            image = os.path.join(image_path2,image_name)
            # print(image_path1)
            # print(image_path2)
            # print(image)
            label_path = image_path1.replace("rawdata","labeldata")
            label_name = os.listdir(label_path)
            # print(label_name)
            # Check if the list is empty
            if not label_name:
                print(f"No files found in {label_path}")
                continue  # Skip to the next iteration of the loop

            label = os.path.join(label_path,str(label_name[0]).replace('[','').replace(']','').replace("'",'').replace("'",''))
            try:
                with open(label, 'rb') as f:
                    text = pickle.load(f)
                    img_dir = image
                    img_format = 'tif'
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    color_check = img_dir.split('\\')[-2]
                    if color_check == 'synthetic':
                        color_space = 'IR'
                    else:
                        color_space = 'SEGMENT'
                    label_dir = label
                    label_format = 'p'
                    camera_3d_center = str(text["camera_3d_center"][count])
                    camera_distance = str(text["camera_distance"][count])
                    iris_loc = str(text["iris_loc"][count])
                    eye_loc = str(text["eye_loc"][count])
                    gaze_angle_az = str(text["gaze_angle_az"][count])
                    gaze_angle_el = str(text["gaze_angle_el"][count])
                    ortho_scale = str(text["ortho_scale"][count])
                    cam_az = str(text["cam_az"][count])
                    cam_el = str(text["cam_el"][count])
                    glasses = str(text["glasses"][count])
                    pupil = str(text["pupil"][count])
                    eye_lid = str(text["eye_lid"][count])
                    iris_rot = str(text["iris_rot"][count])
                    left_corner = str(text["left_corner"][count])
                    right_corner = str(text["right_corner"][count])
                    sclera = str(text["sclera"][count])

                    img_dir = img_dir.replace('Z:', 'DataBase').replace('\\','/')
                    label_dir = label_dir.replace('Z:', 'DataBase').replace('\\','/')

                    value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, camera_3d_center, camera_distance, iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
                    glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera)
                    value_list1.append(value)

                    count += 1

            except Exception as ex:
                error_list.append((label, str(ex)))
                print(f"Error loading data from file: {label}")
                print(f"Error details: {ex}")

                # query 실행
            if len(value_list1) >= 300:
                # print(value_list1[0])
                doUT.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(f'{count} / {len(data_names)}')

        count = 0

doUT.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

insert 300 done
300 / 24
insert 300 done
600 / 24
insert 300 done
900 / 24
insert 300 done
1200 / 24
insert 300 done
1500 / 24
insert 300 done
1800 / 24
Error loading data from file: Z:\RIT_Eyes_s_general\data\labeldata\1\1-general.p
Error details: 'NoneType' object has no attribute 'shape'
insert 300 done
100 / 24
insert 300 done
400 / 24
insert 300 done
700 / 24
insert 300 done
1000 / 24
insert 300 done
1300 / 24
insert 300 done
1600 / 24
insert 300 done
1900 / 24
Error loading data from file: Z:\RIT_Eyes_s_general\data\labeldata\1\1-general.p
Error details: 'NoneType' object has no attribute 'shape'
insert 300 done
200 / 24
insert 300 done
500 / 24
insert 300 done
800 / 24
insert 300 done
1100 / 24
insert 300 done
1400 / 24
insert 300 done
1700 / 24
insert 300 done
2000 / 24
Error loading data from file: Z:\RIT_Eyes_s_general\data\labeldata\1\1-general.p
Error details: 'NoneType' object has no attribute 'shape'
insert 300 done
300 / 24
insert 300 done
600 / 24
insert 300 done
900 / 

## query 작성

In [7]:
query1 = (f'''CREATE TABLE IF NOT EXISTS RIT_Eyes_s_natural_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        camera_3d_center VARCHAR(200) NOT NULL,
        camera_distance VARCHAR(200) NOT NULL,
        iris_loc VARCHAR(200) NOT NULL,
        eye_loc VARCHAR(200) NOT NULL,
        gaze_angle_az VARCHAR(200) NOT NULL,
        gaze_angle_el VARCHAR(200) NOT NULL,
        ortho_scale VARCHAR(200) NOT NULL,
        cam_az VARCHAR(200) NOT NULL,
        cam_el VARCHAR(200) NOT NULL,
        glasses VARCHAR(200) NOT NULL,
        pupil VARCHAR(200) NOT NULL,
        eye_lid VARCHAR(200) NOT NULL,
        iris_rot VARCHAR(200) NOT NULL,
        left_corner VARCHAR(200) NOT NULL,
        right_corner VARCHAR(200) NOT NULL,
        sclera VARCHAR(200) NOT NULL)''')

In [8]:
query2 = (f'''INSERT INTO RIT_Eyes_s_natural_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, camera_3d_center, camera_distance,
          iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
          glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

## query 실행

In [9]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [10]:
# label 경로
label_path = 'Z:\\RIT_Eyes_s_natural\\data\\labeldata'
image_path = 'Z:\\RIT_Eyes_s_natural\\data\\rawdata'

value_list1 = []
error_list = []
count = 0

data_names = os.listdir(image_path)
for data_name in data_names:
    image_path1 = os.path.join(image_path,data_name) 
    for accesory in os.listdir(image_path1):
        image_path2 = os.path.join(image_path1,accesory)
        for image_name in os.listdir(image_path2):
            image = os.path.join(image_path2,image_name)
            # print(image_path1)
            # print(image_path2)
            # print(image)
            label_path = image_path1.replace("rawdata","labeldata")
            label_name = os.listdir(label_path)
            # print(label_name)
            # Check if the list is empty
            if not label_name:
                print(f"No files found in {label_path}")
                continue  # Skip to the next iteration of the loop

            label = os.path.join(label_path,str(label_name[0]).replace('[','').replace(']','').replace("'",'').replace("'",''))
            try:
                with open(label, 'rb') as f:
                    text = pickle.load(f)
                    img_dir = image
                    img_format = 'tif'
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    color_check = img_dir.split('\\')[-2]
                    if color_check == 'synthetic':
                        color_space = 'IR'
                    else:
                        color_space = 'SEGMENT'
                    label_dir = label
                    label_format = 'p'
                    camera_3d_center = str(text["camera_3d_center"][count])
                    camera_distance = str(text["camera_distance"][count])
                    iris_loc = str(text["iris_loc"][count])
                    eye_loc = str(text["eye_loc"][count])
                    gaze_angle_az = str(text["gaze_angle_az"][count])
                    gaze_angle_el = str(text["gaze_angle_el"][count])
                    ortho_scale = str(text["ortho_scale"][count])
                    cam_az = str(text["cam_az"][count])
                    cam_el = str(text["cam_el"][count])
                    glasses = str(text["glasses"][count])
                    pupil = str(text["pupil"][count])
                    eye_lid = str(text["eye_lid"][count])
                    iris_rot = str(text["iris_rot"][count])
                    left_corner = str(text["left_corner"][count])
                    right_corner = str(text["right_corner"][count])
                    sclera = str(text["sclera"][count])

                    img_dir = img_dir.replace('Z:', 'DataBase')
                    label_dir = label_dir.replace('Z:', 'DataBase')

                    value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, camera_3d_center, camera_distance, iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
                    glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera)
                    value_list1.append(value)

                    count += 1

            except Exception as ex:
                error_list.append((label, str(ex)))
                print(f"Error loading data from file: {label}")
                print(f"Error details: {ex}")

                # query 실행
            if len(value_list1) >= 300:
                # print(value_list1[0])
                doUT.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(f'{len(data_name)} / {len(data_names)}')

        count = 0

doUT.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 done
2 / 24
insert 300 

## query 작성

In [11]:
query1 = (f'''CREATE TABLE IF NOT EXISTS RIT_Eyes_s_nvgaze_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        camera_3d_center VARCHAR(200) NOT NULL,
        camera_distance VARCHAR(200) NOT NULL,
        iris_loc VARCHAR(200) NOT NULL,
        eye_loc VARCHAR(200) NOT NULL,
        gaze_angle_az VARCHAR(200) NOT NULL,
        gaze_angle_el VARCHAR(200) NOT NULL,
        ortho_scale VARCHAR(200) NOT NULL,
        cam_az VARCHAR(200) NOT NULL,
        cam_el VARCHAR(200) NOT NULL,
        glasses VARCHAR(200) NOT NULL,
        pupil VARCHAR(200) NOT NULL,
        eye_lid VARCHAR(200) NOT NULL,
        iris_rot VARCHAR(200) NOT NULL,
        left_corner VARCHAR(200) NOT NULL,
        right_corner VARCHAR(200) NOT NULL,
        sclera VARCHAR(200) NOT NULL)''')

In [12]:
query2 = (f'''INSERT INTO RIT_Eyes_s_nvgaze_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, camera_3d_center, camera_distance,
          iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
          glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

## query 실행

In [13]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [14]:
# label 경로
label_path = 'Z:\\RIT_Eyes_s_nvgaze\\data\\labeldata'
image_path = 'Z:\\RIT_Eyes_s_nvgaze\\data\\rawdata'

value_list1 = []
error_list = []
count = 0

data_names = os.listdir(image_path)
for data_name in data_names:
    image_path1 = os.path.join(image_path,data_name) 
    for accesory in os.listdir(image_path1):
        image_path2 = os.path.join(image_path1,accesory)
        for image_name in os.listdir(image_path2):
            image = os.path.join(image_path2,image_name)
            # print(image_path1)
            # print(image_path2)
            # print(image)
            label_path = image_path1.replace("rawdata","labeldata")
            label_name = os.listdir(label_path)
            # print(label_name)
            # Check if the list is empty
            if not label_name:
                print(f"No files found in {label_path}")
                continue  # Skip to the next iteration of the loop

            label = os.path.join(label_path,str(label_name[0]).replace('[','').replace(']','').replace("'",'').replace("'",''))
            try:
                with open(label, 'rb') as f:
                    text = pickle.load(f)
                    img_dir = image
                    img_format = 'tif'
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    color_check = img_dir.split('\\')[-2]
                    if color_check == 'synthetic':
                        color_space = 'IR'
                    else:
                        color_space = 'SEGMENT'
                    label_dir = label
                    label_format = 'p'
                    camera_3d_center = str(text["camera_3d_center"][count])
                    camera_distance = str(text["camera_distance"][count])
                    iris_loc = str(text["iris_loc"][count])
                    eye_loc = str(text["eye_loc"][count])
                    gaze_angle_az = str(text["gaze_angle_az"][count])
                    gaze_angle_el = str(text["gaze_angle_el"][count])
                    ortho_scale = str(text["ortho_scale"][count])
                    cam_az = str(text["cam_az"][count])
                    cam_el = str(text["cam_el"][count])
                    glasses = str(text["glasses"][count])
                    pupil = str(text["pupil"][count])
                    eye_lid = str(text["eye_lid"][count])
                    iris_rot = str(text["iris_rot"][count])
                    left_corner = str(text["left_corner"][count])
                    right_corner = str(text["right_corner"][count])
                    sclera = str(text["sclera"][count])

                    img_dir = img_dir.replace('Z:', 'DataBase')
                    label_dir = label_dir.replace('Z:', 'DataBase')

                    value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, camera_3d_center, camera_distance, iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
                    glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera)
                    value_list1.append(value)

                    count += 1

            except Exception as ex:
                error_list.append((label, str(ex)))
                print(f"Error loading data from file: {label}")
                print(f"Error details: {ex}")

                # query 실행
            if len(value_list1) >= 300:
                print(value_list1[0])
                doUT.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(f'{len(data_name)} / {len(data_names)}')

        count = 0

doUT.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

('DataBase\\RIT_Eyes_s_nvgaze\\data\\rawdata\\1\\mask-withoutskin\\0000.tif', 'tif', 640, 480, 'SEGMENT', 'DataBase\\RIT_Eyes_s_nvgaze\\data\\labeldata\\1\\1-snvgaze.p', 'p', '[-0.017064335517883306, -5.02645169052124, -0.08398039662551879]', '3.8038271432595505', '[0.03500239923596382, -1.0023376941680908, 0.010153090581297874]', '[0.0, 0.0, 0.0]', '1.9999997710638548', '0.5799966692718073', '3.8028616905212402', '0', '90', '0', '0.7630667036738225', '0.0', '[-1.5707963705062866, 0.0, 1.046067476272583]', '[1.1374797821044922, -0.36298900842666626, -0.11673785746097565]', '[-0.7808918356895447, -0.9031707048416138, -0.17035934329032898]', '[0.0, 0.0, 6.098071575164795]')
insert 300 done
1 / 23
('DataBase\\RIT_Eyes_s_nvgaze\\data\\rawdata\\1\\mask-withoutskin\\0300.tif', 'tif', 640, 480, 'SEGMENT', 'DataBase\\RIT_Eyes_s_nvgaze\\data\\labeldata\\1\\1-snvgaze.p', 'p', '[-0.019101935200691228, -5.635785205688476, -0.4980442079277039]', '4.440256687408862', '[-0.25153762102127075, -0.90701

## query 작성

In [15]:
query1 = (f'''CREATE TABLE IF NOT EXISTS RIT_Eyes_s_openeds_info (
        
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        camera_3d_center VARCHAR(200) NOT NULL,
        camera_distance VARCHAR(200) NOT NULL,
        iris_loc VARCHAR(200) NOT NULL,
        eye_loc VARCHAR(200) NOT NULL,
        gaze_angle_az VARCHAR(200) NOT NULL,
        gaze_angle_el VARCHAR(200) NOT NULL,
        ortho_scale VARCHAR(200) NOT NULL,
        cam_az VARCHAR(200) NOT NULL,
        cam_el VARCHAR(200) NOT NULL,
        glasses VARCHAR(200) NOT NULL,
        pupil VARCHAR(200) NOT NULL,
        eye_lid VARCHAR(200) NOT NULL,
        iris_rot VARCHAR(200) NOT NULL,
        left_corner VARCHAR(200) NOT NULL,
        right_corner VARCHAR(200) NOT NULL,
        sclera VARCHAR(200) NOT NULL)''')

In [16]:
query2 = (f'''INSERT INTO RIT_Eyes_s_openeds_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, camera_3d_center, camera_distance,
          iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
          glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

## query 실행

In [17]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [18]:
# label 경로
label_path = 'Z:\\RIT_Eyes_s_openeds\\data\\labeldata'
image_path = 'Z:\\RIT_Eyes_s_openeds\\data\\rawdata'

value_list1 = []
error_list = []
count = 0

data_names = os.listdir(image_path)
for data_name in data_names:
    image_path1 = os.path.join(image_path,data_name) 
    for accesory in os.listdir(image_path1):
        image_path2 = os.path.join(image_path1,accesory)
        for image_name in os.listdir(image_path2):
            image = os.path.join(image_path2,image_name)
            # print(image_path1)
            # print(image_path2)
            # print(image)
            label_path = image_path1.replace("rawdata","labeldata")
            label_name = os.listdir(label_path)
            # print(label_name)
            # Check if the list is empty
            if not label_name:
                print(f"No files found in {label_path}")
                continue  # Skip to the next iteration of the loop

            label = os.path.join(label_path,str(label_name[0]).replace('[','').replace(']','').replace("'",'').replace("'",''))
            try:
                with open(label, 'rb') as f:
                    text = pickle.load(f)
                    img_dir = image
                    img_format = 'tif'
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    color_check = img_dir.split('\\')[-2]
                    if color_check == 'synthetic':
                        color_space = 'IR'
                    else:
                        color_space = 'SEGMENT'
                    label_dir = label
                    label_format = 'p'
                    camera_3d_center = str(text["camera_3d_center"][count])
                    camera_distance = str(text["camera_distance"][count])
                    iris_loc = str(text["iris_loc"][count])
                    eye_loc = str(text["eye_loc"][count])
                    gaze_angle_az = str(text["gaze_angle_az"][count])
                    gaze_angle_el = str(text["gaze_angle_el"][count])
                    ortho_scale = str(text["ortho_scale"][count])
                    cam_az = str(text["cam_az"][count])
                    cam_el = str(text["cam_el"][count])
                    glasses = str(text["glasses"][count])
                    pupil = str(text["pupil"][count])
                    eye_lid = str(text["eye_lid"][count])
                    iris_rot = str(text["iris_rot"][count])
                    left_corner = str(text["left_corner"][count])
                    right_corner = str(text["right_corner"][count])
                    sclera = str(text["sclera"][count])

                    img_dir = img_dir.replace('Z:', 'DataBase')
                    label_dir = label_dir.replace('Z:', 'DataBase')

                    value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, camera_3d_center, camera_distance, iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
                    glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera)
                    value_list1.append(value)

                    count += 1

            except Exception as ex:
                error_list.append((label, str(ex)))
                print(f"Error loading data from file: {label}")
                print(f"Error details: {ex}")

                # query 실행
            if len(value_list1) >= 300:
                # print(value_list1[0])
                doUT.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(f'{len(data_name)} / {len(data_names)}')

        count = 0

doUT.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
insert 300 done
1 / 24
Error loading data from file: Z:\RIT_Eyes_s_openeds\data\labeldata\10\10-sopeneds.p
Error details: 'NoneType' object has no attribute 'shape'
Error loading data from file: Z:\RIT_Eyes_s_openeds\data\labeldata\10\10-sopeneds.p
Error details: 'NoneType' object has no attribute 'shape'
insert 300 done
2 / 24
Error loading data from file: Z:\RIT_Eyes_s_openeds\data\labeldata\10\10-sopeneds.p
Error details: 'NoneType' object has no attribute 'shape'
Error loading data from file: Z:\RIT_Eyes_s_o